## Top Level Domain Report

In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'

# show lots of data
pd.options.display.max_rows=10000

In [2]:
# Event counts for top level domains by event type
# limited to W pages with external links
# not limited to sampled pageloads
w_tld_query = """
SELECT REGEXP_EXTRACT(parse_url(link_url,'HOST'),'(\.[^\.]+)$',1) as tld, action, COUNT(*) AS count 
FROM citationusage 
WHERE wiki = 'enwiki'
AND page_id IN 
        (SELECT DISTINCT page_id 
        FROM ryanmax.pages_with_extlinks
        WHERE to_date(dt) >= '{}' 
        AND to_date(dt) <= '{}'
        )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY tld, action
ORDER BY COUNT(*) DESC
"""
w_tld_events = spark.sql(
    w_tld_query.format(
        start_date_string, end_date_string,
        event_exclusion_sql, start_date_string, end_date_string
    ))
w_tld_events_rdd = w_tld_events.rdd
w_tld_events_df = sqlContext.createDataFrame(w_tld_events_rdd)
w_tld_events_pandas = w_tld_events_df.toPandas()

In [3]:
tld_pda = w_tld_events_pandas.copy()
parsing_errors = tld_pda.loc[(tld_pda['tld'] == '') | (tld_pda['tld'].isnull())]
count_parsing_errors = parsing_errors['count'].sum()
total = tld_pda['count'].sum()
display(Markdown("**Table I**: Count of events by top level domain. W pages w/ external links.  Limited to >= 1000 events."))
display(Markdown("Could not parse hostname/domain from {0} link_urls (first two rows) which represents {1:.2%} of all values.".format(count_parsing_errors,count_parsing_errors/total)))

# limit to counts of 1K or more
df_filtered = tld_pda.query('count>1000').copy()
# set precision before pivot
df_filtered['count'] = df_filtered['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered.pivot(index='tld', columns='action', values='count')

**Table I**: Count of events by top level domain. W pages w/ external links.  Limited to >= 1000 events.

Could not parse hostname/domain from 55105 link_urls (first two rows) which represents 0.06% of all values.

action,extClick,fnClick,fnHover,upClick
tld,,,,
NaN,12409,18212,22268,None
,1496,None,None,None
.122,3411,None,None,None
.132,2203,None,None,None
.173,2992,None,None,None
.194,1907,None,None,None
.27,3142,None,None,None
.ac,1575,None,None,None
.ad,1103,None,None,None


In [4]:
# Event counts for top level domains by event type
# limited to WP:M pages with external links
# not limited to sampled pageloads
wpm_tld_query = """
SELECT REGEXP_EXTRACT(parse_url(link_url,'HOST'),'(\.[^\.]+)$',1) as tld, action, COUNT(*) AS count 
FROM citationusage 
WHERE wiki = 'enwiki'
AND page_id IN 
        (SELECT DISTINCT page_id 
        FROM ryanmax.projmed_with_extlinks
        WHERE to_date(dt) >= '{}' 
        AND to_date(dt) <= '{}'
        )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY tld, action
ORDER BY COUNT(*) DESC
"""
wpm_tld_events = spark.sql(
    wpm_tld_query.format(
        start_date_string, end_date_string,
        event_exclusion_sql, start_date_string, end_date_string
    ))
wpm_tld_events_rdd = wpm_tld_events.rdd
wpm_tld_events_df = sqlContext.createDataFrame(wpm_tld_events_rdd)
wpm_tld_events_pandas = wpm_tld_events_df.toPandas()

In [5]:
tld_pda = wpm_tld_events_pandas.copy()
parsing_errors = tld_pda.loc[(tld_pda['tld'] == '') | (tld_pda['tld'].isnull())]
count_parsing_errors = parsing_errors['count'].sum()
total = tld_pda['count'].sum()
display(Markdown("**Table II**: Count of events by top level domain. WP:M pages w/ external links.  Limited to >= 100 events."))
display(Markdown("Could not parse hostname/domain from {0} link_urls (first row) which represents {1:.2%} of all values.".format(count_parsing_errors,count_parsing_errors/total)))

# limit to counts of 1K or more
df_filtered = tld_pda.query('count>100').copy()
# set precision before pivot
df_filtered['count'] = df_filtered['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered.pivot(index='tld', columns='action', values='count')

**Table II**: Count of events by top level domain. WP:M pages w/ external links.  Limited to >= 100 events.

Could not parse hostname/domain from 951 link_urls (first row) which represents 0.04% of all values.

action,extClick,fnClick,fnHover,upClick
tld,,,,
NaN,102,273,510,None
.al,132,None,None,None
.at,219,None,None,None
.au,5711,None,None,None
.be,386,None,None,None
.br,323,None,None,None
.ca,9971,None,None,None
.ch,980,None,None,None
.cn,849,None,None,None


### .gov domain detail

In [6]:
# Event counts for .gov top level domain by event type
# limited to W pages with external links
# not limited to sampled pageloads
w_gov_query = """
SELECT REGEXP_EXTRACT(parse_url(link_url,'HOST'),'(\.[^\.]+\.[^\.]+)$',1) as domain, action, COUNT(*) AS count 
FROM citationusage 
WHERE wiki = 'enwiki'
AND page_id IN 
        (SELECT DISTINCT page_id 
        FROM ryanmax.pages_with_extlinks
        WHERE to_date(dt) >= '{}' 
        AND to_date(dt) <= '{}'
        )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
AND REGEXP_EXTRACT(parse_url(link_url,'HOST'),'(\.[^\.]+)$',1) = '.gov'
GROUP BY domain, action
ORDER BY COUNT(*) DESC
"""
w_gov_events = spark.sql(
    w_gov_query.format(
        start_date_string, end_date_string,
        event_exclusion_sql, start_date_string, end_date_string
    ))
w_gov_events_rdd = w_gov_events.rdd
w_gov_events_df = sqlContext.createDataFrame(w_gov_events_rdd)
w_gov_events_pandas = w_gov_events_df.toPandas()

In [7]:
display(Markdown("**Table III**: Count of events for .gov domain. W pages w/ external links. Only extClick events found."))
tld_pda = w_gov_events_pandas.copy()
tld_pda.sort_values(by=['count'],ascending=False)[['domain','count']]


**Table III**: Count of events for .gov domain. W pages w/ external links. Only extClick events found.

,domain,count
0,.nih.gov,231437
1,.nps.gov,54546
2,.nasa.gov,43591
3,.loc.gov,36051
4,.census.gov,26097
5,.usgs.gov,25518
6,.ca.gov,23075
7,.house.gov,22679
8,.usda.gov,17724
9,.congress.gov,17248


In [8]:
# Event counts for .gov top level domain by event type
# limited to WP:M pages with external links
# not limited to sampled pageloads
wpm_gov_query = """
SELECT REGEXP_EXTRACT(parse_url(link_url,'HOST'),'(\.[^\.]+\.[^\.]+)$',1) as domain, action, COUNT(*) AS count 
FROM citationusage 
WHERE wiki = 'enwiki'
AND page_id IN 
        (SELECT DISTINCT page_id 
        FROM ryanmax.projmed_with_extlinks
        WHERE to_date(dt) >= '{}' 
        AND to_date(dt) <= '{}'
        )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
AND REGEXP_EXTRACT(parse_url(link_url,'HOST'),'(\.[^\.]+)$',1) = '.gov'
GROUP BY domain, action
ORDER BY COUNT(*) DESC
"""
wpm_gov_events = spark.sql(
    wpm_gov_query.format(
        start_date_string, end_date_string,
        event_exclusion_sql, start_date_string, end_date_string
    ))
wpm_gov_events_rdd = wpm_gov_events.rdd
wpm_gov_events_df = sqlContext.createDataFrame(wpm_gov_events_rdd)
wpm_gov_events_pandas = wpm_gov_events_df.toPandas()

In [9]:
display(Markdown("**Table IV**: Count of events for .gov domain. WP:M pages w/ external links. Only extClick events found."))
wpm_gov_events_pandas.sort_values(by=['count'],ascending=False)[['domain','count']]

**Table IV**: Count of events for .gov domain. WP:M pages w/ external links. Only extClick events found.

,domain,count
0,.nih.gov,89464
1,.cdc.gov,8072
2,.fda.gov,7039
3,.cancer.gov,2771
4,medlineplus.gov,1763
5,.hhs.gov,1143
6,.cms.gov,921
7,.epa.gov,798
8,.gpo.gov,602
9,.usda.gov,428
